In [1]:
import pandas as pd
import sklearn
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, average_precision_score
import numpy as np
from keras.callbacks import Callback

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def read_data(path, train_path, test_path):
    train_data = pd.read_csv(path + train_path)
    test_data = pd.read_csv(path + test_path)
    train_data['_id'] = train_data['_id'].apply(lambda x: x + '.jpg')
    test_data['_id'] = test_data['_id'].apply(lambda x: x + '.jpg')
    return train_data, test_data    

In [3]:
path = "/kaggle/input/pizza-image-error-classification/"
train_path = "pizzacam/pizza_train.csv"
test_path = "pizzacam/pizza_test.csv"
train_data, test_data = read_data(path, train_path, test_path)

In [4]:
test_data.head()

,_id,image_url,Cháy_Baking-Burnt,Khôngđốm_Baking-Doesnothaveleopard-spotting,Màunhạt_Baking-Pale,Lênmenthiếu_Fermentation-Lackfermentation,Lênmenquánhiều_Fermentation-Overfermentation,Viềnkđều_Edge-Uneven,Viềnnhỏ_Edge-Toosmall,Viềnto_Edge-Toobig,...,Quátậptrung_Topping-Toppingtoocentered,Topping-Khôngđều_Noteven,Khônghìnhtròn_Topping-Notcircled,Thiếubóng_toofewballs,Nởviềnkhôngđủ_edgepizzaisnotenoughswollen,Bánhkhôngtròn_Distortedshape,Bánhbé_Size-Toosmall,Bánhlớn_Size-Toobig,Phômaicao_Topping-Cheesetoohigh,image_error
0,644a51aae280ef002847f69e.jpg,https://oven-bucket.pizza4ps.io/images/2023042...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,644a5224e280ef002847f6a1.jpg,https://oven-bucket.pizza4ps.io/images/2023042...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,644a52d0e280ef002847f6ab.jpg,https://oven-bucket.pizza4ps.io/images/2023042...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,644a5370e280ef002847f6b0.jpg,https://oven-bucket.pizza4ps.io/images/2023042...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,644a5381e280ef002847f6b2.jpg,https://oven-bucket.pizza4ps.io/images/2023042...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
train_data.shape

(65221, 26)

In [6]:
test_data.shape

(16305, 26)

In [7]:
train_data.head()

,_id,image_url,Cháy_Baking-Burnt,Khôngđốm_Baking-Doesnothaveleopard-spotting,Màunhạt_Baking-Pale,Lênmenthiếu_Fermentation-Lackfermentation,Lênmenquánhiều_Fermentation-Overfermentation,Viềnkđều_Edge-Uneven,Viềnnhỏ_Edge-Toosmall,Viềnto_Edge-Toobig,...,Quátậptrung_Topping-Toppingtoocentered,Topping-Khôngđều_Noteven,Khônghìnhtròn_Topping-Notcircled,Thiếubóng_toofewballs,Nởviềnkhôngđủ_edgepizzaisnotenoughswollen,Bánhkhôngtròn_Distortedshape,Bánhbé_Size-Toosmall,Bánhlớn_Size-Toobig,Phômaicao_Topping-Cheesetoohigh,image_error
0,647db9d9ba8c270029f76b6f.jpg,https://oven-bucket.pizza4ps.io/images/2023060...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,64636724ba8c270029f70623.jpg,https://oven-bucket.pizza4ps.io/images/2023051...,0,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,64a0028ca2f5430028b12eb0.jpg,https://oven-bucket.pizza4ps.io/images/2023070...,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,64830e48ba8c270029f77a3e.jpg,https://oven-bucket.pizza4ps.io/images/2023060...,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,644b9d31ba8c270029f697a7.jpg,https://oven-bucket.pizza4ps.io/images/2023042...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [2]:
labels = [
    "Cháy_Baking-Burnt",
    "Khôngđốm_Baking-Doesnothaveleopard-spotting",
    "Màunhạt_Baking-Pale",
    "Lênmenthiếu_Fermentation-Lackfermentation",
    "Viềnkđều_Edge-Uneven",
    "Viềnnhỏ_Edge-Toosmall",
    "Viềnto_Edge-Toobig",
    "Viềnthấp_Edge-Toolow",
    "Quátheoviền_Topping-Toostrongshapingtheedge",
    "Topping-Khôngđúng_incorrectportioning",
    "KhôngcânTopping-Notevenhalfandhalf",
    "Sốttrênmép_Topping-Saucecoveringontheedge",
    "Trộnlẫn_Topping-Bended",
    "Quátậptrung_Topping-Toppingtoocentered",
    "Topping-Khôngđều_Noteven",
    "Thiếubóng_toofewballs",
    "Nởviềnkhôngđủ_edgepizzaisnotenoughswollen",
    "Bánhkhôngtròn_Distortedshape",
    "Bánhbé_Size-Toosmall",
    "Phômaicao_Topping-Cheesetoohigh"
]
num_labels = len(labels)
print(num_labels)

20


In [9]:
from keras.preprocessing.image import ImageDataGenerator
image_data_generator = ImageDataGenerator(
#     rotation_range=30,  # Góc xoay ảnh
    rescale=1./255,  # Tiền xử lý: chia màu cho 255 để đưa về khoảng [0, 1]
    # shear_range=0.2,  # Góc cắt ảnh
    # horizontal_flip=True,  # Lật ảnh theo chiều ngang
    # width_shift_range=0.2,  # Dịch ảnh theo chiều ngang
    # height_shift_range=0.2,  # Dịch ảnh theo chiều dọc
    validation_split=0.2  # Tỉ lệ dữ liệu dùng để validation
)
test_data_generator = ImageDataGenerator(rescale=1./255)

In [10]:
image_path = path + 'image_input/image_input'

In [11]:
train_generator = image_data_generator.flow_from_dataframe(
    dataframe=train_data,
    directory=image_path,
    x_col='_id',
    y_col=labels,
    subset='training',
    batch_size=32,
    shuffle=True,
    class_mode='raw',
    target_size=(224, 224),
    seed = 42
)
val_generator = image_data_generator.flow_from_dataframe(
    dataframe=train_data,
    directory=image_path,
    x_col='_id',
    y_col=labels,
    subset='validation',
    batch_size=32,
    shuffle=True,
    class_mode='raw',
    target_size=(224, 224),
    seed = 42
)
test_generator = test_data_generator.flow_from_dataframe(
    dataframe=test_data,
    directory=image_path,
    x_col='_id',
    y_col=labels,
    batch_size=32,
    shuffle=True,
    class_mode='raw',
    target_size=(224, 224)
)

Found 52177 validated image filenames.
Found 13044 validated image filenames.
Found 16305 validated image filenames.


In [12]:
train_generator.n

52177

In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K

def inception_v3_create_model():
  base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(1024, activation = 'relu')(x)
  x = Dropout(0.2)(x)
  predictions = Dense(num_labels, activation='sigmoid')(x)

  base_model.trainable = True
  model = Model(inputs=base_model.input, outputs=predictions)

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
  return model

# inception_v3_model = create_model()


In [14]:
class Metrics(Callback):
    def __init__(self, val_generator, num_classes):
        super().__init__()
        self.val_generator = val_generator
        self.num_classes = num_classes

    def on_epoch_end(self, epoch, logs=None):
        # Duyệt qua từng batch và tính toán
        true_labels = []
        predicted_labels = []

        for i in range(len(self.val_generator)):
            images, labels = self.val_generator[i]
            true_labels.extend(labels)

            # Thực hiện dự đoán
            predictions = self.model.predict(images, verbose=0)

            # Chuyển đổi dự đoán thành nhãn binary (0 hoặc 1)
            binary_predictions = (predictions > 0.5).astype(int)
            predicted_labels.extend(binary_predictions)

        # Chuyển đổi danh sách thành numpy arrays
        true_labels = np.array(true_labels)
        predicted_labels = np.array(predicted_labels)

        # Tính precision và recall cho từng nhãn
        per_class_precision = precision_score(true_labels, predicted_labels, average=None, zero_division=1)
        per_class_recall = recall_score(true_labels, predicted_labels, average=None, zero_division=1)

        # Tính AP (Average Precision) cho từng nhãn
        ap_scores = average_precision_score(true_labels, predicted_labels, average=None)

        # Tính mAP (mean Average Precision)
        map_score = np.mean(ap_scores)
        
        per_class_precision = ["{:.2f}".format(precision) for precision in per_class_precision]
        per_class_recall = ["{:.2f}".format(recall) for recall in per_class_recall]
        ap_scores = ["{:.2f}".format(ap) for ap in ap_scores]
        map_score = "{:.2f}".format(map_score)


        # Hiển thị kết quả
        print(f"Epoch: {epoch + 1}")
        print("Precision:", end = "  ")
        for i in range (len(per_class_precision)):
          print(per_class_precision[i], end = "  ")
        print("\nRecall:   ", end = "  ")
        for i in range (len(per_class_recall)):
          print(per_class_recall[i], end = "  ")
        print("\nAP:       ", end = "  ")
        for i in range (len(ap_scores)):
          print(ap_scores[i], end = "  ")
        print()
        print(f"mAP: {map_score}")
        print()

In [5]:
import os
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

def call_back(checkpoint_path):
  early_stopping_callback = EarlyStopping(
      patience=10,  # Số epochs không có cải thiện trước khi dừng huấn luyện
      monitor='val_loss',  # Monitor validation loss
      restore_best_weights=True,  # Khôi phục trọng số của model tại epoch có validation loss tốt nhất
      verbose=1
  )
  checkpoint_dir = os.path.dirname(checkpoint_path)

  checkpoint_callback = ModelCheckpoint(
      filepath=checkpoint_path,
      save_weights_only=True,  # Lưu toàn bộ mô hình, không chỉ là trọng số
      monitor='val_loss',  # Theo dõi giá trị validation loss
      save_best_only=True,  # Lưu chỉ model có đạt được giá trị tốt nhất trên 'monitor'
      mode='min',  # Theo dõi theo chiều giảm của 'monitor'
      verbose=1  # Hiển thị thông báo khi lưu model
  )

  metrics_callback = Metrics(val_generator=val_generator, num_classes=num_labels)
  return early_stopping_callback, checkpoint_callback, metrics_callback


In [28]:
checkpoint_path = "inceptionv3.ckpt"

In [30]:
checkpoint_dir = os.path.dirname(checkpoint_path)

In [29]:
os.path.dirname(checkpoint_path)

''

In [17]:
early_stopping_callback, checkpoint_callback, metrics_callback = call_back(checkpoint_path)

In [21]:
inception_v3_model = inception_v3_create_model()

In [ ]:
hist = inception_v3_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50,
    validation_data = val_generator,
    validation_steps=len(val_generator),
    callbacks=[checkpoint_callback, early_stopping_callback, metrics_callback]
)

Epoch 1/50
1631/1631 [==============================] - ETA: 0s - loss: 0.2747 - binary_accuracy: 0.8826
Epoch 1: val_loss improved from inf to 0.26734, saving model to /kaggle/working/model_saver/inceptionv3.ckpt
Epoch: 1
Precision:  0.60  0.77  0.85  1.00  0.73  0.75  0.59  0.60  0.23  0.53  0.89  1.00  0.46  0.87  1.00  0.74  0.88  0.80  0.51  0.62  
Recall:     0.81  0.68  0.43  0.00  0.40  0.30  0.74  0.22  0.05  0.41  0.08  0.00  0.04  0.14  0.00  0.71  0.21  0.90  0.68  0.52  
AP:         0.54  0.60  0.54  0.02  0.44  0.42  0.50  0.16  0.03  0.25  0.19  0.11  0.04  0.13  0.02  0.62  0.30  0.79  0.41  0.37  
mAP: 0.32

1631/1631 [==============================] - 1478s 876ms/step - loss: 0.2747 - binary_accuracy: 0.8826 - val_loss: 0.2673 - val_binary_accuracy: 0.8853
Epoch 2/50
1631/1631 [==============================] - ETA: 0s - loss: 0.2392 - binary_accuracy: 0.8972
Epoch 2: val_loss did not improve from 0.26734
Epoch: 2
Precision:  0.86  0.52  0.86  0.24  0.73  0.66  0.62  

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [22]:
print(checkpoint_path)

model\inception_v3_model.ckpt


In [31]:
best_model = inception_v3_create_model()

best_model.load_weights(checkpoint_path)

loss, acc = best_model.evaluate(test_generator,steps=len(test_generator) ,verbose=2)

print(f"Loss: {loss}\nBianry_accuracy: {acc}")

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for inceptionv3.ckpt

# Predict

In [ ]:
# Khởi tạo empty arrays để lưu kết quả
true_labels = []
predicted_labels = []

# Duyệt qua từng batch và tính toán
for i in range(len(test_generator)):
    images, labelss = test_generator[i]
    true_labels.extend(labelss)

    # Thực hiện dự đoán
    predictions = best_model.predict(images, verbose = 0)

    # Chuyển đổi dự đoán thành nhãn binary (0 hoặc 1)
    binary_predictions = (predictions > 0.5).astype(int)
    predicted_labels.extend(binary_predictions)

# Chuyển đổi danh sách thành numpy arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Tính precision và recall cho từng nhãn
per_class_precision = precision_score(true_labels, predicted_labels, average=None)
per_class_recall = recall_score(true_labels, predicted_labels, average=None)

# Tính AP (Average Precision) cho từng nhãn
ap_scores = average_precision_score(true_labels, predicted_labels, average=None)

# Tính mAP (mean Average Precision)
map_score = np.mean(ap_scores)

In [ ]:
# Hiển thị kết quả
for label, precision, recall, ap in zip(labels, per_class_precision, per_class_recall, ap_scores):
    print(f"Label: {label}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"AP: {ap}")
    print("-" * 30)

print(f"mAP: {map_score}")

In [ ]:
test_data

In [ ]:
image_path_test = "/kaggle/input/pizza-image-error-classification/image_input/image_input/"
_id_name = "64ad0192a2f5430028b154fc.jpg"

In [ ]:
path = image_path_test + _id_name

In [ ]:
test_id = test_data[test_data['_id'] == _id_name]

In [ ]:
test_id

In [ ]:
real_labels = test_id[labels]

In [ ]:
real_labels = np.array(real_labels)

In [ ]:
print(real_labels)

In [ ]:
real_labels_name = []
for i in range(len(real_labels)):
    real_labels_name.append([])
    for j in range(len(real_labels[i])):
        if real_labels[i][j] == 1:
            real_labels_name[i].append(labels[j])
        
print(real_labels_name)

In [ ]:
def tranfer_image(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path_test, target_size=(224, 224))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = image.reshape((1,) + image.shape)
    image /= 255.0
    return image

result = best_model.predict(image)

In [ ]:
print(result)

In [ ]:
def decode_predictions(predict):
    result = []
    confidence = []
    for i in range(len(predict)):
        result.append([])
        for j in range(len(predict[i])):
            if predict[i][j] > 0.5:
                result[i].append(labels[j])
                confidence.append(round(predict[i][j], 4))
    predict_label, confidence = result[0], confidence
    return predict_label, confidence
            

In [ ]:
predict_label, confidence = decode_predictions(result)

In [ ]:
print(predict_label)

In [ ]:
confidence

In [ ]:
predict_label, confidence = decode_predictions(result)
for i in range (len(predict_label)):
    print(f"Label: {predict_label[i]}   %Confidence: {confidence[i]}")
    


In [ ]:
print(real_labels_name)

In [ ]:
def display_predict(_id):
    path = image_path_test + _id_name
    test_id = test_data[test_data['_id'] == _id_name]
    real_labels = test_id[labels]
    real_labels = np.array(real_labels)
    
    real_labels_name = []
    for i in range(len(real_labels)):
        real_labels_name.append([])
        for j in range(len(real_labels[i])):
            if real_labels[i][j] == 1:
                real_labels_name[i].append(labels[j])

    print(real_labels_name)